In [1]:

pip install reportlab


     ---------------------------------------- 1.9/1.9 MB 11.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install PyPDF2 sentence-transformers


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import PyPDF2
from sentence_transformers import SentenceTransformer, util
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

# Paths to the two PDFs you want to compare
pdf_path1 = "Email1.pdf"
pdf_path2 = "Email2.pdf"

# Extract text from the PDFs
text1 = extract_text_from_pdf(pdf_path1)
text2 = extract_text_from_pdf(pdf_path2)

# Load a pre-trained sentence embeddings model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Generate embeddings for sentences
sentences1 = [sentence.strip() for sentence in text1.split(".") if sentence.strip()]
sentences2 = [sentence.strip() for sentence in text2.split(".") if sentence.strip()]

# Calculate cosine similarities between sentence embeddings
sentence_embeddings1 = model.encode(sentences1, convert_to_tensor=True)
sentence_embeddings2 = model.encode(sentences2, convert_to_tensor=True)

cosine_similarities = util.pytorch_cos_sim(sentence_embeddings1, sentence_embeddings2)

# Create a new PDF and add sentences with highlighted differences
output_pdf_path = "highlighted_differences.pdf"

c = canvas.Canvas(output_pdf_path, pagesize=letter)
y = 700  # Initial y-coordinate for text

for i in range(min(len(sentences1), len(sentences2))):
    similarity = cosine_similarities[i][i].item()  # Compare parallel sentences
    sentence = f"Sentence {i+1}:\n{sentences1[i]}\n{sentences2[i]}\nSimilarity: {similarity:.4f}\n"
    
    # Highlight sentence pairs with similarity less than 1
    if round(similarity,4) < 1.0 :
        c.setFillColorRGB(1, 0, 0)  # Red color for highlighting
        c.drawString(100, y, sentence)
        print("Line no: " , i)
    else:
        c.drawString(100, y, sentence)
    
    y -= 100  # Move to the next line

c.save()
print(f"Highlighted differences saved in {output_pdf_path}")


Line no:  3
Line no:  4
Line no:  6
Line no:  15
Line no:  20
Line no:  29
Line no:  31
Line no:  35
Line no:  38
Highlighted differences saved in highlighted_differences.pdf
